In [ ]:
###Models to do
# models on cleaned data
# models on averages/trend data
# models on new data (eg. PAV, ladder, ohe venue)
    #make sure to include absolute margin when testing and evaluating these
    #keep track in excel?
# lazy predict

In [1]:
import re
import xgboost as xgb
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)
#import torch.nn as nn
#import touch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random import randint
from Gather_AFL_Data import gatherer as gad
#from fdnn import feature_extractor as fex
import skopt
from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score, KFold
from joblib import dump, load

/home/chris/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


2.2.0
2.4.3


In [2]:
import re
import xgboost as xgb
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)
#import torch.nn as nn
#import touch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random import randint
from Gather_AFL_Data import gatherer as gad
#from fdnn import feature_extractor as fex
import skopt
from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold
import pickle
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Activation
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score, KFold

2.2.0
2.4.3


In [3]:
#get headers
#feed this into a bigger function which specifies the amount of games to go through 
def get_headers(n_games):
    headers = ['Round', 'Home_Team', 'Away_Team', 'Venue', 'H_PAV_Sum', 'A_PAV_Sum']
    example_file = pd.read_csv('Data/Fremantle_clean_stats.csv')
    cl_h = example_file.columns
    cl_h = cl_h[:-5]
    ladder_header = ['Ladder Pos_H', 'Form_H', 'Season Wins_H', 'Season Loss_H', 'Season Draw_H']
    headers = [*headers, *ladder_header]
    j = 1
    while j <= n_games:
        for x in cl_h:
            if 'Match_ID' in x or 'Year' in x:
                continue
            x = 'H_'+ x + ' n-' + str(j)
            headers.append(x)
        j = j + 1
    j = 1
    ladder_header = ['Ladder Pos_A', 'Form_A', 'Season Wins_A', 'Season Loss_A', 'Season Draw_A']
    headers = [*headers, *ladder_header]
    while j <= n_games:
        for x in cl_h:
            if 'Match_ID' in x or 'Year' in x:
                continue
            x = 'A_'+ x + ' n-' + str(j)
            headers.append(x)
        j = j + 1
    return headers

In [4]:
def clean_headers(h):
    headers = []
    for x in h:
        if '<' in x or '>' in x:
            x = x.replace('<',"lt_")
            x = x.replace('>', "gt_")
            #print(x)
        headers.append(str(x))
    return headers

In [5]:
def generate_categorical_headers(h):
    cat_var = ['Round', 'Home_Team', 'Away_Team']
    skip = 0
    for x in h:
        if 'Round' in x:
            if (skip == 0):
                skip = 1
                continue
            cat_var.append(x)
            #print(x)
        elif 'Team_against_ID' in x:
            #print(x)
            cat_var.append(x)
        elif 'Venue' in x:
            cat_var.append(x)
    return cat_var

In [6]:
#one hot encode data and transform the X_data
#first redo, find the categorial variables
def ohe_data(x_data, enc, flag,cat_var):
    #data has not been previously one hot encoded
    if (flag == 0):
        #get columns with categorical data and drop from main DF
        categorical_data = x_data[cat_var]
        x_data = x_data.drop(cat_var, axis = 1)
        #define and fit new OHE. Use it on our categorical data by transforming
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        ohe = ohe.fit(categorical_data)
        categorical_data = ohe.transform(categorical_data)
        #get feature names for better labelling
        fn = ohe.get_feature_names(cat_var)
        #make a dataframe with new OHE data and feature names
            #would have been good to have coded it like this for my Masters project...
        categorical_data = pd.DataFrame(categorical_data)
        categorical_data.columns = fn
        #ensure that it won't get cranky about any different indexes(shouldn't be any but just a good check)
        x_data.reset_index(drop=True, inplace=True)
        categorical_data.reset_index(drop=True, inplace=True)
        #concatenate along column axis
        x_data = pd.concat([x_data, categorical_data], axis = 1)
    else:
        #same as above except used already fitted ohe
        categorical_data = x_data[cat_var]
        x_data = x_data.drop(cat_var, axis = 1)
        categorical_data = enc.transform(categorical_data)
        fn = enc.get_feature_names(cat_var)
        categorical_data = pd.DataFrame(categorical_data)
        categorical_data.columns = fn
        x_data.reset_index(drop=True, inplace=True)
        categorical_data.reset_index(drop=True, inplace=True)
        x_data = pd.concat([x_data, categorical_data], axis = 1, ignore_index=True)
        ohe = enc
    return x_data, ohe

In [7]:
#search parameters for best XGB classifier or best XGB regressor

def param_search(x_data, y_label, class_reg):

    def on_step(optim_result):
        """
        Callback meant to view scores after
        each iteration while performing Bayesian
        Optimization in Skopt"""
        score = xgb_bayes_search.best_score_
        print("best score: %s" % score)
        if score >= 0.98:
            print('Interrupting!')
            return True
    X_train, X_test, y_train, y_test = train_test_split(x_data, y_label, test_size=0.1, random_state=157732)
    print("X_train shape: " + str(X_train.shape))
    print("X_test shape: " + str(X_test.shape))
  #  print("y_train shape: " + str(y_train.shape))
  #  print("y_test shape: " + str(y_test.shape))
    space ={'learning_rate': Real(0.01, 1.0, 'log-uniform'),
        'min_child_weight': Integer(0, 10),
        'max_depth': Integer(0, 50),
        'max_delta_step': Integer(0, 20),
        'subsample': Real(0.01, 1.0, 'uniform'),
        'colsample_bytree': Real(0.01, 1.0, 'uniform'),
        'colsample_bylevel': Real(0.01, 1.0, 'uniform'),
        'reg_lambda': Real(1e-9, 1000, 'log-uniform'),
        'reg_alpha': Real(1e-9, 1.0, 'log-uniform'),
        'gamma': Real(1e-9, 0.5, 'log-uniform'),
        'min_child_weight': Integer(0, 5),
        'n_estimators': Integer(50, 200),
        'scale_pos_weight': Real(1e-6, 500, 'log-uniform')}
    if(class_reg == 0):
        xgbclass = xgb.XGBClassifier(random_state=27022013)
    else:
        xgbclass = xgb.XGBRegressor(random_state=27022013)
    xgb_bayes_search = BayesSearchCV(xgbclass, space, n_iter=60, # specify how many iterations
                                    scoring=None, n_jobs=1, cv=10, verbose=1, random_state=42, n_points=12,
                                 refit=True)
  #  kk = np.isinf(X_train)
  #  if True in kk:
  #  	print("aaaaaaa")
  #  kk = np.isinf(y_train)
  #  if True in kk:
  #  	print("reeeeeee")
    try:
        xgb_bayes_search.fit(X_train, y_train.values.ravel(), callback = on_step)
    except:
        xgb_bayes_search.fit(X_train, y_train.values.ravel(), callback = on_step)
  #  print("BEST PARAMS ARE HERE")
 #   print(xgb_bayes_search.best_params_)
    model = xgb_bayes_search.best_estimator_

    return model

In [8]:
#x = all data
#y = label of whether home team or away team won
#m = xgb classifier for winner prediction
#my = labels of margins
    #try absolute values
#mm = xgb regressor for margin prediction
def eval_xgb_games_margins(x, y, m, my, mm):
    results = []
    error = []
    count = 0
    best_w = m
    high_w = 0
    best_m = mm
    high_m = 100
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    print(x)
    for train,test in cv.split(x,y):
       # print(len(train))
        count = count + 1
        print("Split: " + str(count))
        #comment out fit steps for random forest i guess lol
        prediction = m.fit(x.loc[train],y.loc[train]).predict_proba(x.loc[test])
        margin_pred = mm.fit(x.loc[train], my.loc[train])
        print("variables for auroc curve done. Processing fold accuracy + checking best model")
        y_pred = m.predict(x.loc[test])
        #print(y_pred)
        m_pred = mm.predict(x.loc[test])
        print(m_pred)
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y.loc[test], predictions)
        pcent = accuracy * 100.0
        print("The accuracy of this model is" + str(pcent))
        rmse = sqrt(mean_squared_error(m_pred, my.loc[test]))
        print("The rmse of this model is" + str(rmse))
        results.append(pcent)
        error.append(rmse)
        #change the best model to equal current model
        if(pcent > high_w):
            print("found new best classify")
            best_w = m
            high_w = pcent
        if(rmse < high_m):
            print("found best new margin")
            best_m = mm
            high_m = rmse
    print("Best win percentage split = " +str(high_w))
    print("Best margin rmse = "+str(high_m))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    print("Training Testing Margins: %.2f%% (%.2f%%)" % (np.mean(error), np.std(error)))
    return best_w, best_m

In [9]:
#make this a method param later
#n_games = 5
def run_all_models(n_games):
    #load headers and then subsequently categorical headers
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')

    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, na_enc, 0, cv)
    filename = 'Models/ohe_'+str(n_games)+'_no_2021_games.dat'
    pickle.dump(ohe, open(filename, "wb"))
    #reset headers
    feature_names = x_data.columns

    #loads the ylabel matrix,
    y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

    #loads margin as the y_label
    margin_label = pd.read_csv('Data/assembled_margin_ymatrix_no2020'+str(n_games)+'_games.csv')

    print(margin_label.shape)
    print(y_label.shape)
    print(x_data.shape)

    #regex solution which is apparently necessary??
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_data.columns.values]

    #optimise XGBoost model

    #for predicting win
    model = param_search(x_data, y_label, 0)
    #for predicting margin
    #margin_label = abs(margin_label)
    margin_model = param_search(x_data, margin_label, 1)

   # margin_label = abs(margin_label)
    print(margin_label)
    best_xgb_clas, best_xgb_reg = eval_xgb_games_margins(x_data, y_label, model, margin_label, margin_model)

    #save
    pickle.dump(best_xgb_clas, open("Models/best_xgb_clas_no2020_"+str(n_games)+"_games.dat", "wb"))
    pickle.dump(best_xgb_reg, open("Models/best_xgb_reg_no2020_"+str(n_games)+"_games.dat", "wb"))

In [10]:
#as one big script to go through n_games
n_games = [2,3,10]
for n in n_games:
    run_all_models(n)

(1727, 1)
(1727, 1)
(1727, 733)
X_train shape: (1554, 733)
X_test shape: (173, 733)
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   58.0s finished


best score: 0.6492921492921493
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  5.8min finished


best score: 0.6492921492921493
Fitting 10 folds for each of 12 candidates, totalling 120 fits


...

variables for auroc curve done. Processing fold accuracy + checking best model
[-5.0663307e+01 -1.1573309e+00  6.4246984e+00 -1.5046401e+00
 -1.8680904e+01 -8.9271011e+00 -1.9109375e+01 -1.9979273e+01
 -2.0633360e+01 -2.8164385e+01  1.1684551e+01  4.7161331e+00
  3.5064390e+00 -1.3206645e+01 -6.9359856e+00  1.3896693e+01
 -1.0677740e+01 -2.4046541e+01 -3.8098789e+01  1.5127554e+01
 -1.1224387e+01  3.3584000e+01 -1.6677730e+00 -4.4560590e+00
  1.2746339e+01  1.0764902e+01  1.5734634e+01  2.7601852e+01
  1.0028915e+01  1.6363844e+01 -4.4337764e+00  2.4475462e+01
  7.2113199e+00  4.1405445e+01 -1.0486328e+01  1.5346259e+01
  1.5290188e+01  4.2822021e+01 -2.7605460e+00 -5.2751846e+00
  1.3105013e+01  9.5706539e+00  3.5308128e+01 -1.6055208e+01
 -3.0522038e+01 -1.2348662e+01  2.6363397e+00 -3.0074579e+01
 -2.8429334e+00  3.8635421e-01  4.9449879e+01  1.7233961e+01
 -1.4004637e+01  1.8626205e+01 -3.9896059e+00  1.3316734e+01
  6.6243520e+00 -1.1724142e+00  3.1631432e+01  3.3126743e+01
  3.01

/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


variables for auroc curve done. Processing fold accuracy + checking best model
[ 10.611841    -1.3437532  -26.684675    -7.6530914   17.216497
  57.696064    20.10271    -25.475338     0.81147665  16.582895
 -17.852745   -36.039536     2.1739929   20.714243     2.7345197
 -26.430706    -7.2504716  -32.963646    -5.066364     9.523913
  19.304735    13.6379385  -27.037573   -25.796154     3.7596707
  40.452877   -15.432645     4.5453916    7.4312687   27.391636
  38.669094    39.821503    16.898966     9.296032    -6.9629645
   8.77231      9.553026    30.567436    -2.2705166  -27.172419
  12.407394    -7.468421     3.6700625   22.145428    36.194374
  33.40088      2.0090685   -7.0991817  -35.58264     -8.393964
   6.9108534    9.328843    14.205585    49.063473   -22.986366
  38.116352    23.735516    15.822063     4.4178367   29.42389
 -22.208347    27.618542    37.852737   -15.408936    -7.1836514
  48.14827      4.293752     3.2422543   17.290909    30.155949
 -24.518188    21.4569

/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


variables for auroc curve done. Processing fold accuracy + checking best model
[  6.1628733   31.977472   -15.680733    -4.9606485  -15.938332
  37.514248    38.041973     6.197401    10.197627     1.0443907
  26.813412    -5.171053    -5.328283    -0.94594145   8.578268
  24.329847     3.3619924   26.033741    14.357881    20.779425
   4.6384897   14.136189   -30.417765    42.817467   -23.67804
  11.557793    17.85923     35.492096    32.90142     -0.4291826
   6.214527   -25.807142    10.923281    10.730735    32.46081
  33.789913    20.183588    43.74462     -4.7756743  -16.273167
   8.124239   -27.371338    37.225124    -0.72857857  21.729284
  -1.3461937   -8.761271    33.63508     41.201736    22.274544
  -0.0757702    3.8587496  -12.569274    -7.957453    27.373585
  31.354723    17.363234    20.859682   -38.63597    -38.83895
  10.874514    13.192856     4.7514057   25.006529     3.913768
 -10.433552    13.934347    21.043901    -7.934148     0.29476762
 -14.361511   -21.638412

# Best of 3 Models

In [16]:
def prep_data_test(n_games,ohe):
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')
    x_data = x_data.tail(1655)
    
    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, ohe, 1, cv)
    
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_data.columns.values]

    return x_data

In [17]:
def load_test_models(n):
    clf = pickle.load(open("Models/best_xgb_clas_no2020_"+str(n)+"_games.dat", "rb"))
    ohe = pickle.load(open("Models/ohe_"+str(n)+"_no_2021_games.dat", "rb"))
    return clf, ohe

In [18]:
#load data
y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no202010_games.csv')

model_2, ohe = load_test_models(2)
x_2 = prep_data_test(2,ohe)
model_3, ohe = load_test_models(3)
x_3 = prep_data_test(3, ohe)
model_10, ohe = load_test_models(10)
x_10 = prep_data_test(10, ohe)


In [ ]:
results = []
count = 0
high_w = 0
cv = StratifiedKFold(n_splits=10, shuffle=True)
y = y_label
for train,test in cv.split(x_2,y):
    count = count + 1
    print("Split: " + str(count))
    prediction = model_2.fit(x_2.loc[train],y.loc[train]).predict_proba(x_2.loc[test])
    prediction = model_3.fit(x_3.loc[train],y.loc[train]).predict_proba(x_3.loc[test])
    prediction = model_10.fit(x_10.loc[train],y.loc[train]).predict_proba(x_10.loc[test])
    print("variables for auroc curve done. Processing fold accuracy + checking best model")
    d = {'y_pred_2':model_2.predict(x_2.loc[test]), 'y_pred_3':model_3.predict(x_3.loc[test]),
         'y_pred_10':model_10.predict(x_10.loc[test])}
    pred_df = pd.DataFrame(data=d)
    summary_ave_data = pred_df.copy()
    summary_ave_data['average'] = summary_ave_data.mean(numeric_only=True, axis=1)
    print(summary_ave_data)
    y_pred = summary_ave_data["average"].tolist()
    print(y_pred)
    predictions = [round(value) for value in y_pred]
    print(predictions)
    #sees how accurate the model was when testing the test set
    accuracy = accuracy_score(y.loc[test], predictions)
    pcent = accuracy * 100.0
    print("The accuracy of this model is" + str(pcent))
    results.append(pcent)
print("Best win percentage split = " +str(high_w))
print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))

Split: 1


/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

variables for auroc curve done. Processing fold accuracy + checking best model
     y_pred_2  y_pred_3  y_pred_10   average
0         0.0       0.0        0.0  0.000000
1         0.0       1.0        1.0  0.666667
2         0.0       0.0        0.0  0.000000
3         0.0       1.0        0.0  0.333333
4         0.0       0.0        0.0  0.000000
..        ...       ...        ...       ...
161       1.0       1.0        0.0  0.666667
162       0.0       0.0        0.0  0.000000
163       1.0       1.0        1.0  1.000000
164       0.0       0.0        0.0  0.000000
165       1.0       1.0        1.0  1.000000

[166 rows x 4 columns]
[0.0, 0.6666666666666666, 0.0, 0.3333333333333333, 0.0, 0.6666666666666666, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.3333333333333333, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.6666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 0.3333333333333333, 0.0, 0.3333333333333333, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.6666

/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

variables for auroc curve done. Processing fold accuracy + checking best model
     y_pred_2  y_pred_3  y_pred_10   average
0         0.0       0.0        0.0  0.000000
1         1.0       0.0        0.0  0.333333
2         1.0       0.0        0.0  0.333333
3         0.0       0.0        0.0  0.000000
4         1.0       1.0        0.0  0.666667
..        ...       ...        ...       ...
161       0.0       0.0        0.0  0.000000
162       1.0       1.0        1.0  1.000000
163       0.0       1.0        0.0  0.333333
164       1.0       0.0        0.0  0.333333
165       0.0       0.0        0.0  0.000000

[166 rows x 4 columns]
[0.0, 0.3333333333333333, 0.3333333333333333, 0.0, 0.6666666666666666, 0.0, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 1.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0, 1.0, 1.0, 0.0, 1.0, 0.6666666666666666, 0.3333333333333333, 0.0, 0.3333333333333333, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.6666666666666666, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0

/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

# Clean Data Testing Ground

In [11]:
def create_R_TeamDict():
    teams = {
    "1" : "Adelaide Crows",
    "2" : "Brisbane Lions",
    "3" : "Carlton",
    "4" : "Collingwood",
    "5" : "Essendon",
    "6" : "Fremantle",
    "7" : "Geelong Cats",
    "8" : "Gold Coast Suns",
    "9" : "GWS Giants",
    "10": "Hawthorn",
    "11": "Melbourne",
    "12": "North Melbourne",
    "13": "Port Adelaide",
    "14": "Richmond",
    "15": "St Kilda",
    "16": "Sydney Swans",
    "17": "West Coast Eagles",
    "18": "Western Bulldogs"
    }
    return teams

In [157]:
team_int = 6

In [162]:
pavs = pd.read_csv("R_Code/all_team_pavs.csv")
team_pavs = pavs.loc[(pavs["Team_ID"]==team_int)]
team_pavs

,Year,Round,Team_ID,Player_PAV_Total
1012,2013,1,6,233.52
1013,2013,2,6,219.10
1014,2013,3,6,225.28
1015,2013,4,6,204.98
1016,2013,5,6,213.05
...,...,...,...,...
1211,2021,23,6,181.11
1212,2022,1,6,173.39
1213,2022,2,6,168.25
1214,2022,3,6,133.87


In [159]:
df.shape

(204, 100)

In [15]:
def calc_sum_pav(year, rnd, team_int):
    #do calc
    g = gad()
    team_dict = g.createTeamDict()
    r_dict = create_R_TeamDict()
    current_team = (team_dict[str(team_int)])
    df = pd.read_csv("Data/"+current_team+'_clean_stats.csv')
    current_r_team = (r_dict[str(team_int)])
    lineups = pd.read_csv("R_Code/all_lineups.csv")
    lineups = lineups[lineups.isin([current_r_team]).any(axis=1)]
    lineups = lineups[lineups.isin([year]).any(axis=1)]
    lineups = lineups[lineups.isin([rnd]).any(axis=1)]
    lineups['team'] = team_int
    if(rnd < 11):
        lineups['year'] = (year-1)
    lineups.columns = ['year', 'teamname', 'roundNumber', 'firstname', 'surname', 'team']
    cols = ['team', 'year', 'firstname', 'surname']
    lineups = lineups[cols]
    all_pavs = pd.read_csv("R_Code/all_player_PAVs.csv")
    
    lineups.firstname = lineups.firstname.str.split(' ').str[0]
    all_pavs.firstname = all_pavs.firstname.str.replace(' ','')
    
    lineups = lineups.merge(all_pavs, how='inner', on=['team', 'year', 'firstname', 'surname'])
    if(lineups.shape[0] > 0):
        pav = lineups['PAV_total'].sum()
    else:
        pav = 999
    print(pav)
    pav_array = [year, rnd, team_int, pav]
    return pav_array

In [18]:
pav_array = []
for team_id in range(1,19):
    for year in range(2022, 2023):
        for rnd in range(1,5):
            print(str(team_id) + '_' + str(year)+'_'+str(rnd))
            pa = calc_sum_pav(year, rnd, team_id)
            if(pa[3] == 999):
                continue
            pav_array.append(pa)
pav_df = pd.DataFrame(pav_array, columns=['Year', 'Round', 'Team_ID', 'Player_PAV_Total'])
all_pav_df = pd.read_csv('R_Code/all_team_pavs.csv')
all_pav_df = pd.concat([all_pav_df, pav_df], ignore_index=True)
print(pav_df)
print(all_pav_df)
all_pav_df.to_csv("R_Code/all_team_pavs.csv", header=True, index=False)

1_2022_1
156.19000000000003
1_2022_2
144.26000000000002
1_2022_3
153.15000000000003
1_2022_4
174.11000000000004
2_2022_1
265.28000000000003
2_2022_2
256.13
2_2022_3
265.28
2_2022_4
250.97000000000003
3_2022_1
156.67000000000002
3_2022_2
178.84000000000003
3_2022_3
157.47
3_2022_4
169.10999999999999
4_2022_1
177.64000000000001
4_2022_2
191.04000000000002
4_2022_3
201.59
4_2022_4
195.71
5_2022_1
214.8
5_2022_2
230.61999999999995
5_2022_3
203.24
5_2022_4
191.28000000000003
6_2022_1
173.39
6_2022_2
168.25
6_2022_3
133.87
6_2022_4
166.42000000000002
7_2022_1
236.32000000000002
7_2022_2
251.10000000000002
7_2022_3
249.79
7_2022_4
218.20999999999998
8_2022_1
185.58
8_2022_2
177.15
8_2022_3
182.41
8_2022_4
176.84
9_2022_1
206.89000000000001
9_2022_2
200.72
9_2022_3
190.59
9_2022_4
182.04
10_2022_1
149.82
10_2022_2
166.4
10_2022_3
151.77999999999997
10_2022_4
145.51
11_2022_1
279.64
11_2022_2
273.71999999999997
11_2022_3
267.33
11_2022_4
282.88
12_2022_1
197.87
12_2022_2
189.13
12_2022_3
177.45

# Trend Based Dataset Testing

In [98]:
#get assemble_df code from the clean_dataset csv
#do the usual stuff appended at the front eg. venue, ladder, round, H_team_id, A_team_id
#pick averageable statistics and get n_game average for home and away
#function to workout from clean_dataset home_team wins against away team in last 5
    #5-home_team_wins to get away team wins against oppo
    #append to new X_data
#save to new assemble_df.csv

match_id = 10576
team_id = 9
oppo_id = 6
g = gad()
teams = g.createTeamDict()
n_games = 3


In [135]:
def find_form(ct, ot, match_id):
    g = gad()
    teams = g.createTeamDict()
    current_team = (teams[str(ct)])
    team_string = current_team+"_clean_stats.csv"
    df = pd.read_csv("Data/"+team_string)
    idx = df.index[df['Match_ID'] == match_id]
    my_idx = idx[0]
    df = df.loc[0:my_idx-1,:]
    prev_games = df.loc[(df['Team_against_ID']==ot)]
    #return last 5 games against each other and 8th column is whether the ct won against ot
    l = 5
    if(prev_games.shape[0] < 5):
        l = prev_games.shape[0]
    if(l==0):
        wins = 0
    else:
        prev_games = prev_games.iloc[-l: , 8]
        wins = prev_games.sum()
    return wins

In [130]:
def create_prev_games(match_id, team_id, teams, n_games, oppo_id):
    margin = None
    ma = None
    y_label = None
    trend_cg = []
    current_team = (teams[str(team_id)])
    print(current_team)
    print(match_id)
    team_string = current_team+"_clean_stats.csv"
    df = pd.read_csv("Data/"+team_string)
    #drops ladder stats
    #finds where in the dataframe the current match is
    idx = df.index[df['Match_ID'] == match_id]
    #print(idx)
    my_idx = idx[0]
    #splits dataframe into game data and end of round ladder data
    l_df = df.iloc[:,-5:]
    t_df = df.iloc[: , :-5]
    current_year = t_df.loc[my_idx][1]
    if(current_year == 2020.0):
        print('game in 2020')
        margin = 8888
    else:
        #turns the WWWLL form column into # of W
        n_form = []
        for x in l_df['form']:
            if(len(x)<n_games):
                y=float(x.count("W"))
                n_form.append(y)
            else:
                x=x[-n_games:]
                y=float(x.count("W"))
                n_form.append(y)
        l_df['form'] = n_form
        #checks to make sure there is enough games to go through
        if(my_idx < (n_games)):
            print('Num of Prev Games Exceeds previous games')
            margin = 9999
        else:
            #start match array with the ladder values from end of previous round (as this would be current for predicting round)
            ma = l_df.loc[my_idx-1].values
            #finds both labels for models
            y_label = t_df.loc[my_idx]["H/A Win?"]
            margin = t_df.loc[my_idx]["Margin"]
            #start from the previous game to current game
            #i is to know how many games included
            i = 1
            #j finds the previous game and allows for 2020 exclusion
            j = 1
            while i <= n_games:
                year = t_df.loc[my_idx-j][1]
                if(year == 2020.0):
                    j = j + 1
                    continue
                cg = t_df.loc[my_idx-j][2:].values
                trend_cg.append(cg)
                i = i + 1
                j = j + 1
            my_df = pd.DataFrame(trend_cg, columns = t_df.columns[2:])
            my_df = my_df.iloc[:,3:-1]
            cg = my_df.mean(axis=0)
            ma = [*ma, *cg]
            h_form = find_form(team_id, oppo_id, match_id)
            o_form = find_form(oppo_id, team_id, match_id)
            form_diff = h_form-o_form
            ma.append(form_diff)
    return ma, y_label, margin

In [136]:
home_array, y_label, margin = create_prev_games(5720, team_id, teams, 3, oppo_id)

GWS
5720
18
0
0
0


In [110]:
len(home_array)

95

In [107]:
headers = ['Round', 'Home_Team', 'Away_Team', 'Venue']
example_file = pd.read_csv('Data/Fremantle_clean_stats.csv')
cl_h = example_file.columns
cl_h = cl_h[5:-6]
ladder_header = ['Ladder Pos_H', 'Form_H', 'Season Wins_H', 'Season Loss_H', 'Season Draw_H']
headers = [*headers, *ladder_header]
for x in cl_h:
    if 'Match_ID' in x or 'Year' in x:
        continue
    x = 'H_'+ x + ' avg'
    headers.append(x)
headers.append("H_Recent_Matchup_Diff")
ladder_header = ['Ladder Pos_A', 'Form_A', 'Season Wins_A', 'Season Loss_A', 'Season Draw_A']
headers = [*headers, *ladder_header]
for x in cl_h:
    if 'Match_ID' in x or 'Year' in x:
        continue
    x = 'A_'+ x + ' avg'
    headers.append(x)
headers.append("A_Recent_Matchup_Diff")
len(headers)

194

# Team Specific Models
## Averaged about 60%

In [121]:
def get_team_specific_dataset(n_games, tn):
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()
    #load in x_data and y_label for n = 1 (quicker)
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')
    y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

    #finds team index
    team_idx = x_data.index[(x_data['Home_Team']==tn) | (x_data['Away_Team']==tn)]
    team_x_data = x_data.iloc[team_idx]
    team_y_data = y_label.iloc[team_idx]
    team_x_data.reset_index(drop=True, inplace=True)
    team_y_data.reset_index(drop=True, inplace=True)

    #quick shape check
    print(team_x_data.shape)
    print(team_y_data.shape)
    l = team_x_data.shape[0]

    #loop for new y_array
    i = 0
    new_team_y = []
    while i < l:
        x = team_x_data.iloc[i]
        y = team_y_data.iloc[i]
        #if our team for current model is the home team
        if(x["Home_Team"] == tn):
            #and the home team has won
            if(y["H/A Win?"] == 0):
                #set win val to 1
                w_val = 1
            #else they're the home team but the away team won
            else:
                #set win val to 0
                w_val = 0
        #else they're the away team
        else:
            #if the home team won, that means they lost
            if(y["H/A Win?"] == 0):
                w_val = 0
            #else they're the away team and they won
            else:
                w_val = 1
        i = i + 1
        new_team_y.append(w_val)
    ny = pd.DataFrame(new_team_y, columns=['Team Won?'])
    return team_x_data, ny

In [116]:
x, y = get_team_specific_dataset(10, 1)

(176, 1874)
(176, 1)


,Team Won?
0,0
1,0
2,1
3,0
4,0
...,...
171,0
172,0
173,0
174,1


In [136]:
def eval_team_xgb_games(x, y, m, n_games, tn):
    results = []
    error = []
    count = 0
    best_w = m
    high_w = 0
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    for train,test in cv.split(x,y):
       # print(len(train))
        count = count + 1
        print("Split: " + str(count))
        #comment out fit steps for random forest i guess lol
        prediction = m.fit(x.loc[train],y.loc[train]).predict_proba(x.loc[test])
        y_pred = m.predict(x.loc[test])
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y.loc[test], predictions)
        pcent = accuracy * 100.0
        results.append(pcent)
        #change the best model to equal current model
        if(pcent > high_w):
            best_w = m
            high_w = pcent
    print("Training Testing Accuracy for n_games="+str(n_games)+" team="+str(tn)+" : %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best_w, np.mean(results)

In [132]:
def run_team_models(n_games, tn):
    #load headers and then subsequently categorical headers
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data, y_label = get_team_specific_dataset(n_games, tn)

    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, na_enc, 0, cv)
    filename = 'Models/Team_OHE/ohe_'+str(n_games)+"_team_specific_"+str(tn)+'_no_2020_games.dat'
    pickle.dump(ohe, open(filename, "wb"))
    #reset headers
    feature_names = x_data.columns

    #regex solution which is apparently necessary??
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_data.columns.values]

    #optimise XGBoost model
    #print(x_data)
    #for predicting win
    model = param_search(x_data, y_label, 0)
    best_xgb_clas, model_av = eval_team_xgb_games(x_data, y_label, model, n_games, tn)

    #save
    pickle.dump(best_xgb_clas, open("Models/Team_Models/best_xgb_clas_no2020_"+str(n_games)+"_team_specific_"+str(tn)+'_.dat', "wb"))
    return model_av

In [138]:
import warnings
warnings.filterwarnings('ignore')
for n in range(1,11):
    averages = []
    for tn in range(1,19):
        a = run_team_models(n, tn)
        averages.append(a)
    print("Accuracy for Team Models for n_games= "+str(n)+" : %.2f%% (%.2f%%)" % (np.mean(averages), np.std(averages)))

(185, 200)
(185, 1)
X_train shape: (166, 392)
X_test shape: (19, 392)
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    8.2s finished


best score: 0.5602409638554217
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    6.7s finished


best score: 0.5602409638554217
Fitting 10 folds for each of 12 candidates, totalling 120 fits


...

best score: 0.6172839506172839
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   58.3s finished


best score: 0.6172839506172839
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   58.4s finished


best score: 0.6172839506172839
Split: 1
Split: 2
Split: 3
Split: 4
Split: 5
Split: 6
Split: 7
Split: 8
Split: 9
Split: 10
Training Testing Accuracy for n_games=10 team=18 : 54.44% (5.44%)
Accuracy for Team Models for n_games= 10 : 59.12% (7.12%)


# Nearest Centroid

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from Gather_AFL_Data import gatherer as gad
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
import pickle
from sklearn.neighbors import NearestCentroid
import re
from numpy import arange
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.naive_bayes import BernoulliNB

In [6]:
def load_data(n_games):
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')

    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, na_enc, 0, cv)

    #reset headers
    feature_names = x_data.columns

    #loads the ylabel matrix,
    y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

    #loads margin as the y_label
    margin_label = pd.read_csv('Data/assembled_margin_ymatrix_no2020'+str(n_games)+'_games.csv')

    print(margin_label.shape)
    print(y_label.shape)
    print(x_data.shape)

    #regex solution which is apparently necessary??
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_data.columns.values]
    return x_data, y_label

In [ ]:
def params_NC(x_data, y_label):
# define model
    print("optimising hyperparameters")
    model = NearestCentroid()
    # define model evaluation method
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # define grid
    grid = dict()
    grid['shrink_threshold'] = arange(0, 1.01, 0.01)
    grid['metric'] = ['euclidean', 'manhattan']
    # define search
    search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
    # perform the search
    results = search.fit(x_data, y_label.values.ravel())
    # summarize
    print('Mean Accuracy: %.3f' % results.best_score_)
    print('Config: %s' % results.best_params_)
    return results.best_params_

In [ ]:
def eval_NC(x, y, m):
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    results = []
    print(x)
    count = 0
    best_w = m
    high_w = 0
    for train,test in cv.split(x,y):
        count = count + 1
        print("Split: " + str(count))
        x_train = x.loc[train]
        y_train = y.loc[train].values.ravel()
        x_test = x.loc[test]
        y_test = y.loc[test]
        m.fit(x_train,y_train)
        y_pred = m.predict(x_test)
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y_test, predictions)
        pcent = accuracy * 100.0
        results.append(pcent)
        if(pcent > high_w):
            print("found new best classify")
            best_w = m
            high_w = pcent
    print("Best win percentage split = " +str(high_w))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best_w

In [ ]:
def run_all_NC(n):
    x_data, y_label = load_data(n)
    best_params = params_NC(x_data, y_label)
    model = NearestCentroid(**best_params)
    best_NC_clas = eval_NC(x_data, y_label, model)
    pickle.dump(best_NC_clas, open("Models/best_NC_clas_no2020"+str(n_games)+'_games.sav', 'wb'))

In [ ]:
n_games = [1,2,3,4,5,6,7,10]
for n in n_games:
    run_all_NC(n)

# BernoulliNB

In [23]:
def params_BNB(x_data, y_label):
# define model
    print("optimising hyperparameters")
    model = BernoulliNB()
    # define model evaluation method
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # define grid
    grid = dict()
    grid['alpha'] = arange(0.01, 1.01, 0.01)
    # define search
    search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=1, verbose=1)
    # perform the search
    results = search.fit(x_data, y_label.values.ravel())
    # summarize
    print('Mean Accuracy: %.3f' % results.best_score_)
    print('Config: %s' % results.best_params_)
    return results.best_params_

In [18]:
def eval_BNB(x, y, m):
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    results = []
    print(x)
    count = 0
    best_w = m
    high_w = 0
    for train,test in cv.split(x,y):
        count = count + 1
        print("Split: " + str(count))
        x_train = x.loc[train]
        y_train = y.loc[train].values.ravel()
        x_test = x.loc[test]
        y_test = y.loc[test]
        m.fit(x_train,y_train)
        y_pred = m.predict(x_test)
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y_test, predictions)
        pcent = accuracy * 100.0
        results.append(pcent)
        if(pcent > high_w):
            print("found new best classify")
            best_w = m
            high_w = pcent
    print("Best win percentage split = " +str(high_w))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best_w

In [19]:
def run_all_BNB(n):
    x_data, y_label = load_data(n)
    best_params = params_BNB(x_data, y_label)
    model = BernoulliNB(**best_params)
    best_BNB_clas = eval_BNB(x_data, y_label, model)
    pickle.dump(best_BNB_clas, open("Models/best_BNB_clas_no2020"+str(n_games)+'_games.sav', 'wb'))

In [24]:
n_games = [1,2,3,4,5,6,7,10]
for n in n_games:
    run_all_BNB(n)

(1646, 1)
(1646, 1)
(1646, 409)
optimising hyperparameters
Fitting 30 folds for each of 100 candidates, totalling 3000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 3000 out of 3000 | elapsed:   56.7s finished


Mean Accuracy: 0.602
Config: {'alpha': 0.89}
      Ladder Pos_H  Form_H  Season Wins_H  Season Loss_H  Season Draw_H  \
0             12.0     0.0            0.0            1.0            0.0   
1              4.0     1.0            1.0            0.0            0.0   
2              2.0     1.0            1.0            0.0            0.0   
3             17.0     0.0            0.0            1.0            0.0   
4              1.0     1.0            1.0            0.0            0.0   
...            ...     ...            ...            ...            ...   
1641           3.0     0.0           16.0            6.0            0.0   
1642           4.0     1.0           15.0            7.0            0.0   
1643           1.0     1.0           17.0            4.0            1.0   
1644           2.0     1.0           17.0            5.0            0.0   
1645           1.0     1.0           17.0            4.0            1.0   

      H_H/A? n-1  H_H/A Win? n-1  H_Points For n-1  H_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 3000 out of 3000 | elapsed:  1.4min finished


Mean Accuracy: 0.599
Config: {'alpha': 0.7000000000000001}
      Ladder Pos_H  Form_H  Season Wins_H  Season Loss_H  Season Draw_H  \
0              4.0     2.0            2.0            0.0            0.0   
1             12.0     0.0            0.0            2.0            0.0   
2             18.0     0.0            0.0            2.0            0.0   
3             16.0     0.0            0.0            2.0            0.0   
4             11.0     1.0            1.0            1.0            0.0   
...            ...     ...            ...            ...            ...   
1632           3.0     1.0           16.0            6.0            0.0   
1633           4.0     2.0           15.0            7.0            0.0   
1634           1.0     2.0           17.0            4.0            1.0   
1635           2.0     2.0           17.0            5.0            0.0   
1636           1.0     2.0           17.0            4.0            1.0   

      H_H/A? n-1  H_H/A Win? n-1  H_Poin

...

Mean Accuracy: 0.611
Config: {'alpha': 0.9500000000000001}
      Ladder Pos_H  Form_H  Season Wins_H  Season Loss_H  Season Draw_H  \
0              2.0     6.0            6.0            0.0            0.0   
1              3.0     5.0            5.0            1.0            0.0   
2             14.0     2.0            2.0            4.0            0.0   
3             16.0     1.0            1.0            5.0            0.0   
4              6.0     4.0            4.0            2.0            0.0   
...            ...     ...            ...            ...            ...   
1596           3.0     4.0           16.0            6.0            0.0   
1597           4.0     4.0           15.0            7.0            0.0   
1598           1.0     4.0           17.0            4.0            1.0   
1599           2.0     6.0           17.0            5.0            0.0   
1600           1.0     4.0           17.0            4.0            1.0   

      H_H/A? n-1  H_H/A Win? n-1  H_Poin

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 3000 out of 3000 | elapsed:  4.8min finished


Mean Accuracy: 0.608
Config: {'alpha': 1.0}
      Ladder Pos_H  Form_H  Season Wins_H  Season Loss_H  Season Draw_H  \
0             10.0     3.0            3.0            4.0            0.0   
1              2.0     6.0            6.0            1.0            0.0   
2              3.0     6.0            6.0            1.0            0.0   
3             13.0     3.0            3.0            4.0            0.0   
4              5.0     5.0            5.0            2.0            0.0   
...            ...     ...            ...            ...            ...   
1587           3.0     5.0           16.0            6.0            0.0   
1588           4.0     4.0           15.0            7.0            0.0   
1589           1.0     5.0           17.0            4.0            1.0   
1590           2.0     6.0           17.0            5.0            0.0   
1591           1.0     5.0           17.0            4.0            1.0   

      H_H/A? n-1  H_H/A Win? n-1  H_Points For n-1  H_P

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 3000 out of 3000 | elapsed:  6.8min finished


Mean Accuracy: 0.603
Config: {'alpha': 0.060000000000000005}
      Ladder Pos_H  Form_H  Season Wins_H  Season Loss_H  Season Draw_H  \
0              5.0     7.0            7.0            3.0            0.0   
1             18.0     0.0            0.0           10.0            0.0   
2              9.0     5.0            5.0            5.0            0.0   
3             13.0     4.0            4.0            6.0            0.0   
4             16.0     2.0            2.0            8.0            0.0   
...            ...     ...            ...            ...            ...   
1560           3.0     7.0           16.0            6.0            0.0   
1561           4.0     7.0           15.0            7.0            0.0   
1562           1.0     6.0           17.0            4.0            1.0   
1563           2.0     9.0           17.0            5.0            0.0   
1564           1.0     6.0           17.0            4.0            1.0   

      H_H/A? n-1  H_H/A Win? n-1  H_Po

# LAZY PREDICT

In [ ]:
import pandas as pd
import numpy as np
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.model_selection import train_test_split
from Gather_AFL_Data import gatherer as gad
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
import pickle

In [ ]:
def eval_lazy_predict(x, y, my):
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    all_clf_models = []
    all_reg_models = []
    count = 0
    for train,test in cv.split(x,y):
       # print(len(train))
        count = count + 1
        print("Split: " + str(count))
        #classifier
        X_train = x.loc[train]
        X_test = x.loc[test]
        y_train = y.loc[train]
        y_test = y.loc[test]
        my_train = my.loc[train]
        my_test = my.loc[test]
        clf = LazyClassifier(predictions=True)
        reg = LazyRegressor(predictions=True)
        if(count == 1):
            all_clf_models, predictions = clf.fit(X_train, X_test, y_train, y_test)
          #  all_reg_models, reg_predictions = reg.fit(X_train, X_test, my_train, my_test)
            all_clf_models.sort_index(inplace=True)
           # all_reg_models.sort_index(inplace=True)
            #print(all_models)
        else:
            new_clf_models, predictions = clf.fit(X_train, X_test, y_train, y_test)
           # new_reg_models, reg_predictions = reg.fit(X_train, X_test, my_train, my_test)
            new_clf_models.sort_index(inplace=True)
           # new_reg_models.sort_index(inplace=True)
            #print(new_models)
            all_clf_models = all_clf_models.add(new_clf_models)
           # all_reg_models = all_reg_models.add(new_reg_models)
    return all_clf_models#, all_reg_models

In [ ]:
def run_all_lp(n_games):
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')

    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, na_enc, 0, cv)

    #reset headers
    feature_names = x_data.columns

    #loads the ylabel matrix,
    y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

    #do margins in a second when this works
    margin_label = pd.read_csv('Data/assembled_margin_ymatrix_no2020'+str(n_games)+'_games.csv')
    margin_label = abs(margin_label)

    clf_lp = eval_lazy_predict(x_data, y_label, margin_label)

    clf_lp.sort_values(by='Accuracy', ascending=False, inplace=True)
    #reg_lp.sort_values(by='RMSE', inplace=True)
    pct_lp = clf_lp*10
    print(pct_lp)

In [ ]:
#as one big script to go through n_games
n_games = [1,2,3,4,5,6,7,10]
for n in n_games:
    run_all_lp(n)

# Deep Learningggg

In [ ]:
def build_DNN_model(x_len):
    model = Sequential()
    model.add(Dense(63, input_dim = x_len))
    model.add(Activation('relu'))
    model.add(Dropout(0.03))
    model.add(BatchNormalization())

    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.02))

    model.add(Dense(16))
    model.add(Activation('relu'))

    model.add(Dense(8))
    model.add(Activation('relu'))
    #add output layer
    model.add(Dense(1, activation='linear'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)

    model.compile(loss="mean_squared_error", optimizer=opt, metrics=[tf.keras.metrics.RootMeanSquaredError()])
    print(model.summary())
    return model

In [ ]:
def build_CNN_model(x_len):
    #del model
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=14,
                     input_shape=(x_len, 1)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(filters=16, kernel_size=10,
                     input_shape=(32, 1)))
    model.add(Activation('linear'))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=10, kernel_size=8,
                     input_shape=(16, 1)))
    model.add(Activation('linear'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(64, activation='linear'))
    model.add(Dense(32, activation='linear'))
    model.add(Dense(16, activation='linear'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='linear'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"


    model.compile(loss="mean_squared_error", optimizer=opt, metrics=[tf.keras.metrics.RootMeanSquaredError()])

    print(model.summary())
    return model

In [ ]:
#flag = 0 (DNN)
#flag = 1 (CNN)
def eval_dl(x,y,k,flag):
    cv = StratifiedKFold(n_splits=k,shuffle=True)
    best_model = []
    results = []
    highest = 0
    i = 1
    for train,test in cv.split(x,y):
        if(flag == 0):
            model = build_DNN_model(x[train].shape[1])
        if(flag == 1):
            x = x.reshape(x.shape[0], x.shape[1], 1)
            model = build_CNN_model(x[train].shape[1])
        bs = ((x[train].shape[0])/20)
        bs = round(bs)
        history = model.fit(x[train], y[train], validation_data=(x[test], y[test]), epochs = 50, batch_size=bs)
        _, accuracy = model.evaluate(x[test], y[test], batch_size=bs, verbose=0)
        accuracy = accuracy * 100
        print("accuracy for model " + str(i) + " is " + str(accuracy))
        if(accuracy > highest):
            highest = accuracy
            best_model = model
        results.append(accuracy)
        i = i + 1
    print("highest accuracy is: " + str(highest))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best_model